## PyCall.jl 

* Embed a python implementation within the Julia process
* Call python functions and methods with reasonable overhead
* Zero copy arrays from Julia to Python
* Uses `numpy` arrays

###Install

* `Pkg.add("PyCall")`
* PyCall needs to find a "good" C-Python implementation 
* Anaconda python is your best bet on windows
* On OSX, Homebrew python is good. Default installed python does not work
* Read the docs/src for troubleshooting issues in finding python

###Usage

* `@pyimport` to load python modules
* The module name is then available as a Julia value
* call functions directly on the modules
* Julia values of basic types are automatically converted to python values


In [2]:
using PyCall
@pyimport math
math.sqrt(pi/2) - sqrt(pi / 2)  

0.0

In [4]:
@pyimport Bio.Seq as s
@pyimport Bio.Alphabet as a
my_dna = s.Seq("AGTACACTGGT", a.generic_dna)

PyObject Seq('AGTACACTGGT', DNAAlphabet())

In [5]:
my_dna[:find]("ACT")

5

###Arrays

* Arrays of julia bits types are passed without copying

In [7]:
@pyimport numpy
A=[4,5,2,3,7,9,1]
numpy.sort(A)

7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 7
 9

In [8]:
A 

7-element Array{Int64,1}:
 4
 5
 2
 3
 7
 9
 1

### Internals

* `PyObject` is a type that stores references to python object
* `pycall` takes a callable python object, and calls it with the supplied arguments

### Calling object methods

* . 
* . is not overloadable in Julia
* replacement is indexing by method name 
* PyObject[:name]

In [12]:
PyObject(A)[:sort]()
A

7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 7
 9

In [17]:
numpy.random[:rand](10)

10-element Array{Float64,1}:
 0.883106
 0.197137
 0.564138
 0.59404 
 0.106377
 0.369958
 0.411774
 0.7743  
 0.181168
 0.6524  

In [25]:
@pyimport numpy.random as nr
nr.rand(10)  

10-element Array{Float64,1}:
 0.0847289
 0.621008 
 0.3663   
 0.728217 
 0.107494 
 0.717853 
 0.450123 
 0.0493813
 0.969083 
 0.446928 

In [22]:
typeof(numpy.random[:rand])

Function

In [23]:
typeof(numpy.random["rand"])

PyObject (constructor with 34 methods)

In [26]:
r=pycall(numpy.random["rand"], PyVector, 10)

10-element Any PyVector:
 0.522688 
 0.865097 
 0.170235 
 0.501857 
 0.102898 
 0.448367 
 0.0434687
 0.263296 
 0.132502 
 0.717742 

In [20]:
r[1] 

0.6874488152811445

### Julia functions can be passed to python functions that accept a lambda

In [4]:
@pyimport scipy.optimize as so
so.newton(cos, 1) - pi/2

0.0

In [38]:
so.newton(x -> cos(x) - x, 1)

0.7390851332151607